In [1]:
###############################################################################
# The Institute for the Design of Advanced Energy Systems Integrated Platform
# Framework (IDAES IP) was produced under the DOE Institute for the
# Design of Advanced Energy Systems (IDAES).
#
# Copyright (c) 2018-2023 by the software owners: The Regents of the
# University of California, through Lawrence Berkeley National Laboratory,
# National Technology & Engineering Solutions of Sandia, LLC, Carnegie Mellon
# University, West Virginia University Research Corporation, et al.
# All rights reserved.  Please see the files COPYRIGHT.md and LICENSE.md
# for full copyright and license information.
###############################################################################

# Pump Unit Model with IAPWS Property Package
Author: Anuja Deshpande  
Maintainer: Brandon Paul  
Updated: 2023-06-01  



<img src="pump.svg" alt="drawing" width="200"/>



## Learning Outcomes

- Demonstrate use of the Pump unit model in IDAES
- Demonstrate different simulation options available


## Problem Statement

In this example, we will pump a stream of water at atmospheric pressure to a pressure of 201325 Pa using a simple pump unit model. We will be using the IAPWS property package for the water properties. It is assumed that the pump operates at steady state.

The inlet specifications are as follows:

* Flow Rate = 100 mol/s
* Mole fraction (H2O) = 1
* Pressure = 101325 Pa
* Temperature = 298.15 K

We will simulate 2 different cases, depending on the operating specifications by the user:

**Case 1**:In this case, we will specify the pump efficiency and the pressure increase variable on the pump unit model.

   * Pressure Increase = 100000 Pa
   * Pump Efficiency = 0.8
        
**Case 2**: In this case, we will specify the pump efficiency but will specify the pressure of the outlet stream directly. 
   * Outlet Pressure = 201325 Pa
   * Pump efficiency = 0.8

IDAES documentation reference for pump model: https://idaes-pse.readthedocs.io/en/stable/

## Setting up the problem in IDAES

In the following cell, we will be importing the necessary components from Pyomo and IDAES.

In [2]:
# Import objects from pyomo package
from pyomo.environ import ConcreteModel, SolverFactory, value, units

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

# Create the ConcreteModel and the FlowsheetBlock, and attach the flowsheet block to it.
m = ConcreteModel()

m.fs = FlowsheetBlock(
    dynamic=False
)  # dynamic or ss flowsheet needs to be specified here


# Import the IAPWS property package to create a properties block for the flowsheet
from idaes.models.properties import iapws95
from idaes.models.properties.helmholtz.helmholtz import PhaseType

# Add properties parameter block to the flowsheet with specifications
m.fs.properties = iapws95.Iapws95ParameterBlock(phase_presentation=PhaseType.L)

## Case 1: Fix pressure change and pump efficiency 


### Add Pump Unit Model

In [3]:
# Import pump unit model from the model library
from idaes.models.unit_models.pressure_changer import Pump

# Create an instance of the pump unit, attaching it to the flowsheet
# Specify that the property package to be used with the pump is the one we created earlier.
m.fs.pump_case_1 = Pump(property_package=m.fs.properties)

In [4]:
# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
# DOF = Number of Model Variables - Number of Model Constraints
from idaes.core.util.model_statistics import degrees_of_freedom

# Call the degrees_of_freedom function, get initial DOF
DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

The initial DOF is 5


### Fix Inlet Stream Conditions

In [5]:
# Fix the stream inlet conditions
m.fs.pump_case_1.inlet.flow_mol[0].fix(100)  # mol/s

# Use the htpx method to obtain the molar enthalpy of inlet stream based on given conditions of temperature and pressure
m.fs.pump_case_1.inlet.enth_mol[0].fix(
    value(iapws95.htpx(T=298.15 * units.K, P=101325 * units.Pa))
)  # J/mol
m.fs.pump_case_1.inlet.pressure[0].fix(101325)  # Pa

### Fix Pressure Change and Pump Efficiency

In [6]:
# Fix pump conditions
m.fs.pump_case_1.deltaP.fix(100000)
m.fs.pump_case_1.efficiency_pump.fix(0.8)

# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m)
print("The final DOF is {0}".format(DOF_final))

The final DOF is 0


### Initialization

In [7]:
# Initialize the flowsheet, and set the logger level to INFO
m.fs.pump_case_1.initialize(outlvl=idaeslog.INFO)

2023-11-02 10:30:15 [INFO] idaes.init.fs.pump_case_1.control_volume: Initialization Complete


2023-11-02 10:30:15 [INFO] idaes.init.fs.pump_case_1: Initialization Complete: optimal - Optimal Solution Found


### Solve Model

In [8]:
# Solve the simulation using ipopt
# Note: If the degrees of freedom = 0, we have a square problem
opt = SolverFactory("ipopt")
solve_status = opt.solve(m, tee=True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://



### View Results

In [9]:
# Display a readable report
m.fs.pump_case_1.report()


Unit : fs.pump_case_1                                                      Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key             : Value      : Units         : Fixed : Bounds
         Efficiency :    0.80000 : dimensionless :  True : (None, None)
    Mechanical Work :     225.85 :          watt : False : (None, None)
    Pressure Change : 1.0000e+05 :        pascal :  True : (None, None)
     Pressure Ratio :     1.9869 : dimensionless : False : (None, None)

------------------------------------------------------------------------------------
    Stream Table


                         Units           Inlet     Outlet  
    Molar Flow          mole / second     100.00     100.00
    Mass Flow       kilogram / second     1.8015     1.8015
    T                          kelvin     298.15     298.16
    P                          pascal 1.0132e+05 2.0132e+05
    Vapor Fraction      dimensionless     0.0000     0.0000
    Molar Enthalpy       joule / mole     1890.2     1892.4


## Case 2: Fix outlet pressure and pump efficiency


### Add Pump Unit Model

In [10]:
# Create an instance of another pump unit, attaching it to the same flowsheet
# Specify that the property package to be used with the pump is the one we created earlier.
m.fs.pump_case_2 = Pump(property_package=m.fs.properties)

# Call the degrees_of_freedom function, get initial DOF
DOF_initial = degrees_of_freedom(m.fs.pump_case_2)
print("The initial DOF is {0}".format(DOF_initial))

The initial DOF is 5


### Fix Inlet Stream Conditions

In [11]:
# Fix the stream inlet conditions
m.fs.pump_case_2.inlet.flow_mol[0].fix(100)  # mol/s

# Use the htpx method to obtain the molar enthalpy of inlet stream based on given conditions of temperature and pressure
m.fs.pump_case_2.inlet.enth_mol[0].fix(
    value(iapws95.htpx(T=298.15 * units.K, P=101325 * units.Pa))
)  # J/mol
m.fs.pump_case_2.inlet.pressure[0].fix(101325)  # Pa

### Fix Outlet Pressure & Pump Efficiency

In [12]:
# Fix outlet stream conditions
m.fs.pump_case_2.outlet.pressure[0].fix(201325)

# Fix pump efficiency
m.fs.pump_case_2.efficiency_pump.fix(0.8)

In [13]:
DOF_final = degrees_of_freedom(m.fs.pump_case_2)
print("The final degrees of freedom is: {0}".format(DOF_final))

The final degrees of freedom is: 0


### Initialization

In [14]:
# Initialize the flowsheet, and set the logger level to INFO
m.fs.pump_case_2.initialize(outlvl=idaeslog.INFO)

2023-11-02 10:30:15 [INFO] idaes.init.fs.pump_case_2.control_volume: Initialization Complete


2023-11-02 10:30:15 [INFO] idaes.init.fs.pump_case_2: Initialization Complete: optimal - Optimal Solution Found


### Solve Model

In [15]:
# Solve the simulation using ipopt
# Note: If the degrees of freedom = 0, we have a square problem
opt = SolverFactory("ipopt")
solve_status = opt.solve(m.fs.pump_case_2, tee=True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

### View Results

In [16]:
# Display a readable report
m.fs.pump_case_2.report()


Unit : fs.pump_case_2                                                      Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key             : Value      : Units         : Fixed : Bounds
         Efficiency :    0.80000 : dimensionless :  True : (None, None)
    Mechanical Work :     225.85 :          watt : False : (None, None)
    Pressure Change : 1.0000e+05 :        pascal : False : (None, None)
     Pressure Ratio :     1.9869 : dimensionless : False : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                         Units           Inlet     Outlet  
    Molar Flow          mole / second     100.00     100.00
    Mass Flow       kilogram / second     1.8015     1.8015
    T                          kelvin     298.15     298.16
    P                          pascal 1.0132e+05 2.0132e+05
    Vapor Fraction      dimension